In [1]:
### V1, 03.05.2016, Celine ###
####### Extract bval and bvec files from the corresponding method file #######

%matplotlib inline
import re #regular expression
from pandas import DataFrame
import numpy as np

In [13]:
# Replace with your parameters, rest doesnt need changes #
subjectnames = ["P0_F02","P2_F25","P2_F28","P4_F07","P4_F08","P8_F10","P16_F20","P16_F21","P32_F16","P32_F17","P32_F19","P64_F01","P64_F04","P64_F05","P64_F22"] 
subjectpath = "/Users/ghfc/Desktop/Celine/"
methodfilename = "method"

for subjectname in subjectnames:
    getBval(subjectname, subjectpath, methodfilename)

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )

##$PVM_DwEffBval=( 210 )

##$PVM_DwDir=( 200, 3 )



In [9]:
def getBval(subjectname, subjectpath, methodfilename):
    f = open(subjectpath+subjectname+'/'+methodfilename,"r")

    #Extract bvals from 'method' file in a data frame

    for line in f:
        if re.search(r'PVM_DwEffBval=',line): #find parameter of bval
            #print line
            ne = f.next() #read line after the name of the parameter
            nb_str = ne.split()
            bval_arr = [float(x) for x in nb_str] #first line in an array
            ne = f.next()
            while(re.match("[-+]?\d*\.\d+|\d+",ne)!=None): #while the next line is made of floats parse in the array
                nb_str2 = ne.split()
                for x in nb_str2:
                    bval_arr.append(x)
                ne = f.next()
            bval = DataFrame(bval_arr) #array in data frame
            bval=bval.transpose() #transpose data frame to have one line with all the bvals
            bval.to_csv(subjectpath+subjectname+'/'+subjectname+'.bval', sep=" ", index=False, header=False) #save data frame in a file 
    f.close()
    getBvec(subjectname, subjectpath, methodfilename, bval)
    getInfo(subjectname, subjectpath, methodfilename, bval)

In [7]:
def getBvec(subjectname, subjectpath, methodfilename, bval):
    f = open(subjectpath+subjectname+'/'+methodfilename,"r")

    #Extract bvecs from 'method' file in a data frame

    for line in f:
        if re.search(r'PVM_DwDir=',line): #find parameter of bvec
            #print line
            bvec_arr = np.zeros((3,bval.size)) #define the bvec array (3 lines (x,y,z), same number as bval columns)
            i=0
            ne = f.next() #read line after the name of the parameter
            while(re.match("[-+]?\d*\.\d+|\d+",ne)!=None): #while the next line is made of floats parse in the array
                nb_str = ne.split()
                for x in nb_str:
                    j = i%3 #line number (modulo)
                    k = i/3 #column number
                    bvec_arr[j][k+10] = float(x) #assign value in the array (+10 because 10 first B0 are 0)
                    i=i+1
                ne = f.next() #when all floats of the current line are done, go to the next
            bvec = DataFrame(bvec_arr) #array in data frame
            bvec.to_csv(subjectpath+subjectname+'/'+subjectname+'.bvec', sep=" ", index=False, header=False) #save data frame in a file 
    f.close()

In [8]:
def getInfo(subjectname, subjectpath, methodfilename, bval):
    f = open(subjectpath+subjectname+'/'+methodfilename,"r")

    #Get info
    #PVM_Matrix= ==>x, y
    #PVM_SPackArrNSlices= ==>z
    #PVM_SpatResol= ==> resolution in x, y
    #PVM_SPackArrSliceDistance= ==> resolution in z
    #nb bval

    for line in f:
        if re.search(r'PVM_Matrix=',line): 
            ne = f.next()
            nb_str = ne.split()
            mat_x = int(nb_str[1])
            mat_y = int(nb_str[0])
        if re.search(r'PVM_SPackArrNSlices=',line):
            ne = f.next()
            mat_z = int(ne)
        if re.search(r'PVM_SpatResol=',line):
            ne = f.next()
            nb_str = ne.split()
            res_x = float(nb_str[1])
            res_y = float(nb_str[0])
        if re.search(r'PVM_SPackArrSliceDistance=',line):
            ne = f.next()
            res_z = float(ne)
    f.close()

    info_arr = np.zeros((3,3))
    info_arr[0][0] = mat_x
    info_arr[0][1] = mat_y
    info_arr[0][2] = mat_z
    info_arr[1][0] = mat_x * res_x
    info_arr[1][1] = mat_y * res_y
    info_arr[1][2] = mat_z * res_z
    info_arr[2][0] = bval.size

    #print info_arr

    info = DataFrame(info_arr)
    info.to_csv(subjectpath+subjectname+'/'+subjectname+'_info'+'.txt', sep=" ", index=False, header=False) #save data frame in a file 